In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import modules.logger_tool as logger
import os

os.environ['LOG_NAME'] = 'science'
os.environ['LOG_DIR'] = 'logs'
os.environ['LOG_LEVEL'] = 'DEBUG'

logging = logger.get_logger(os.environ['LOG_NAME'], log_level=os.environ['LOG_LEVEL'], log_path=os.environ['LOG_DIR'], log_file=os.environ['LOG_NAME'])


In [ ]:
import modules.database.schemas.curriculum_neo as neo_curriculum
import modules.database.schemas.relationships.curricular_relationships as neo_relationships
import modules.database.tools.xl_planner_tools as planner
import modules.database.tools.neontology_tools as neon
import modules.database.tools.neo4j_driver_tools as driver_tools
import modules.database.tools.neo4j_session_tools as session_tools

import requests
from pydantic import ValidationError


In [ ]:
db_name = 'science'

In [ ]:
url = 'http://localhost:9500/database/admin/stop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

url = 'http://localhost:9500/database/admin/drop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

In [ ]:
url = 'http://localhost:9500/database/admin/create-database'
params = {'db_name': db_name}
response = requests.post(url, params=params)
logging.info(response.text)

In [ ]:
driver = driver_tools.get_driver(database=db_name)


In [ ]:
neon.init_neo4j_connection()


In [ ]:
# Create KeyStageNodes from a list
key_stages = [
    neo_curriculum.KeyStageNode(key_stage_level=3, key_stage='KS3'),
    neo_curriculum.KeyStageNode(key_stage_level=4, key_stage='KS4'),
    neo_curriculum.KeyStageNode(key_stage_level=5, key_stage='KS5')
]
# Now create a routine to create key stages from a list
for key_stage in key_stages:
    neon.create_or_merge_neontology_node(key_stage, database=db_name, operation='create')


In [ ]:
# Create SubjectNodes from a list
subjects = [
    neo_curriculum.SubjectNode(subject_id='SCI', subject_name='Science'),
    neo_curriculum.SubjectNode(subject_id='PHY', subject_name='Physics'),
    neo_curriculum.SubjectNode(subject_id='CHE', subject_name='Chemistry'),
    neo_curriculum.SubjectNode(subject_id='BIO', subject_name='Biology'),
    neo_curriculum.SubjectNode(subject_id='MAT', subject_name='Mathematics'),
    neo_curriculum.SubjectNode(subject_id='ENG', subject_name='English'),
    neo_curriculum.SubjectNode(subject_id='HIS', subject_name='History')
]
# Now create a routine to create subjects from a list
for subject in subjects:
    neon.create_or_merge_neontology_node(subject, database=db_name, operation='create')


In [ ]:
excel_planner = planner.create_dataframes(os.getenv("EXCEL_PLANNER_FILE"))

topic_df = excel_planner['topiclookup_df']
lesson_df = excel_planner['lessonlookup_df']
statement_df = excel_planner['statementlookup_df']


In [ ]:
topic_excel_node_list = []
topic_excel_helper_list = []
topic_lesson_excel_node_list = []
topic_lesson_excel_helper_list = []
learning_statement_excel_node_list = []
learning_statement_excel_helper_list = []

default_topic_values = {
    'topic_assessment_type': 'Null'
    }

default_topic_lesson_values = {
    'topic_lesson_title': 'Null',  # Corrected default value key
    'topic_lesson_type': 'Null',  # Corrected default value key
    'topic_lesson_length': 1,             # Corrected default value key
    'topic_lesson_suggested_activities': 'Null',  # Corrected default value key
    'topic_lesson_skills_learned': 'Null',  # Corrected default value key
    'topic_lesson_weblinks': 'Null',   # Corrected default value key
}

default_learning_statement_values = {
    # Add default values for fields that might contain NaN
    'lesson_learning_statement': 'Null',
    'lesson_learning_statement_type': 'Student learning outcome'
}

# Create a list of TopicNode instances from the topic_df
for index, row in topic_df.iterrows():
    # Filter and map the row to TopicNode fields
    topic_node_data = {
        'topic_id': row.get('TopicID'),
        'topic_title': row.get('TopicTitle'),
        'total_number_of_lessons_for_topic': row.get('TotalNumberOfLessonsForTopic'),
        'topic_type': row.get('TopicType'),
        'topic_assessment_type': row.get('TopicAssessmentType')
    }
    topic_excel_helper_data = {
        'topic_source': row.get('TopicSource'),
        'topic_department': row.get('TopicDepartment'),
        'topic_key_stage': row.get('TopicKeyStage'),
        'topic_year': row.get('TopicYear'),
        'topic_subject': row.get('TopicSubject'),
        'topic_sequence': row.get('TopicSequence')
    }
    # Replace NaN values with defaults
    topic_excel_node_data_processed = planner.replace_nan_with_default(topic_node_data, default_topic_values)
    topic_excel_helper_data_processed = planner.replace_nan_with_default(topic_excel_helper_data, default_topic_values)

    # Create a TopicNode instance for each row
    try:
        topic_node = neo_curriculum.TopicNode(**topic_excel_node_data_processed)
        combined_data = {
            'node': topic_node,
            'helper': topic_excel_helper_data
        }
        topic_excel_node_list.append(combined_data)
    except ValidationError as e:
        logging.error(f"Validation error for row {index}: {e}")

# Create a list of TopicLessonNode instances from the lesson_df
for index, row in lesson_df.iterrows():
    # Filter and map the row to TopicLessonNode fields
    topic_lesson_node_data = {
        'topic_lesson_id': row.get('LessonID'),
        'topic_lesson_title': row.get('LessonTitle', default_topic_lesson_values['topic_lesson_title']),
        'topic_lesson_type': row.get('TopicLessonType', default_topic_lesson_values['topic_lesson_type']),
        'topic_lesson_length': row.get('SuggestedNumberOfPeriodsForLesson', default_topic_lesson_values['topic_lesson_length']),
        'topic_lesson_suggested_activities': row.get('SuggestedActivities', default_topic_lesson_values['topic_lesson_suggested_activities']),
        'topic_lesson_skills_learned': row.get('SkillsLearned', default_topic_lesson_values['topic_lesson_skills_learned']),
        'topic_lesson_weblinks': row.get('TopicLessonWeblinks', default_topic_lesson_values['topic_lesson_weblinks'])
        }
    topic_lesson_excel_helper_data = {
        'topic_lesson_source': row.get('TopicSource'),
        'topic_lesson_department': row.get('TopicDepartment'),
        'topic_lesson_key_stage': row.get('TopicKeyStage'),
        'topic_lesson_topic_id': row.get('TopicID'),
        'topic_lesson_topic_year': row.get('TopicYear'),
        'topic_lesson_topic_subject': row.get('TopicSubject'),
        'topic_lesson_topic_sequence': row.get('TopicSequence'),
        'topic_lesson_lesson_sequence': row.get('LessonSequence'),
        'topic_lesson_learning_objective': row.get('LessonLearningObjective'),
        }

    # Replace NaN values with defaults
    topic_lesson_excel_node_data_processed = planner.replace_nan_with_default(topic_lesson_node_data, default_topic_lesson_values)
    topic_lesson_excel_helper_data_processed = planner.replace_nan_with_default(topic_lesson_excel_helper_data, default_topic_lesson_values)

    # Create a TopicLessonNode instance for each row
    try:
        topic_lesson_node = neo_curriculum.TopicLessonNode(**topic_lesson_excel_node_data_processed)
        combined_data = {
            'node': topic_lesson_node,
            'helper': topic_lesson_excel_helper_data
        }
        topic_lesson_excel_node_list.append(combined_data)
    except ValidationError as e:
        logging.error(f"Validation error for row {index}: {e}")

# Create a list of LearningStatementNode instances from the statement_df
for index, row in statement_df.iterrows():
    # Filter and map the row to LearningStatementNode fields
    learning_statement_node_data = {
        'lesson_learning_statement_id': row.get('StatementID'),
        'lesson_learning_statement': row.get('LearningOutcomeStatement', default_learning_statement_values['lesson_learning_statement']),
        'lesson_learning_statement_type': row.get('LearningStatementType', default_learning_statement_values['lesson_learning_statement_type']),
    }
    learning_statement_excel_helper_data = {
        'lesson_learning_statement_source': row.get('TopicSource'),
        'lesson_learning_statement_department': row.get('TopicDepartment'),
        'lesson_learning_statement_key_stage': row.get('TopicKeyStage'),
        'lesson_learning_statement_topic_id': row.get('TopicID'),
        'lesson_learning_statement_lesson_id': row.get('LessonID'),
        'lesson_learning_statement_topic_year': row.get('TopicYear'),
        'lesson_learning_statement_topic_subject': row.get('TopicSubject'),
        'lesson_learning_statement_topic_sequence': row.get('TopicSequence'),
        'lesson_learning_statement_lesson_sequence': row.get('LessonSequence'),
        'lesson_learning_statement_sequence': row.get('LearningOutcomeSequence')            
    }

    # Replace NaN values with defaults
    learning_statement_excel_node_data_processed = planner.replace_nan_with_default(learning_statement_node_data, default_learning_statement_values)
    learning_statement_excel_helper_data_processed = planner.replace_nan_with_default(learning_statement_excel_helper_data, default_learning_statement_values)

    # Create a LearningStatementNode instance for each row
    try:
        learning_statement_node = neo_curriculum.LearningStatementNode(**learning_statement_excel_node_data_processed)
        combined_data = {
            'node': learning_statement_node,
            'helper': learning_statement_excel_helper_data
        }
        learning_statement_excel_node_list.append(combined_data)
    except ValidationError as e:
        logging.error(f"Validation error for row {index}: {e}")

In [ ]:
# Then, use the create_or_merge_neontology_node function to add these nodes to your Neo4j database
for node_data in topic_excel_node_list:
    try:
        neon.create_or_merge_neontology_node(node_data['node'], database=db_name, operation='merge')
    except Exception as e:
        logging.error(f"Error in processing node: {e}")

for node_data in topic_lesson_excel_node_list:
    try:
        neon.create_or_merge_neontology_node(node_data['node'], database=db_name, operation='create')
    except Exception as e:
        logging.error(f"Error in processing node: {e}")

for node_data in learning_statement_excel_node_list:
    try:
        neon.create_or_merge_neontology_node(node_data['node'], database=db_name, operation='create')
    except Exception as e:
        logging.error(f"Error in processing node: {e}")

In [ ]:
# Create relationships between TopicNode and TopicLessonNode
relationship_list = []
for topic_lesson_node in topic_lesson_excel_node_list:
    topic_lesson_node_id = topic_lesson_node['node'].topic_lesson_id
    with driver.session(database=db_name) as session:
        lesson_nodes = session_tools.find_nodes_by_label_and_properties(session, 'Lesson', {'topic_lesson_id': topic_lesson_node_id})
    if not lesson_nodes:  # Check if the list is empty
        logging.error(f"No lesson node found for ID {topic_lesson_node_id}")
        continue

    topic_node_id = topic_lesson_node['helper']['topic_lesson_topic_id']
    with driver.session(database=db_name) as session:
        topic_nodes = session_tools.find_nodes_by_label_and_properties(session, 'Topic', {'topic_id': topic_node_id})
    if not topic_nodes:  # Check if the list is empty
        logging.error(f"No topic node found for ID {topic_node_id}")
        continue

    logging.debug(f"Lesson node: {lesson_nodes}")
    logging.debug(f"Topic node: {topic_nodes}")
    
    # Assuming only one node is expected for each query
    try:
        lesson_node_properties = lesson_nodes[0]
        topic_node_properties = topic_nodes[0]
    except IndexError:
        logging.error("Error extracting node properties")
        continue

    # Create instances of TopicNode and TopicLessonNode using the extracted properties
    neontology_lesson_node = neo_curriculum.TopicLessonNode(**lesson_node_properties)
    neontology_topic_node = neo_curriculum.TopicNode(**topic_node_properties)

    # Create the relationship
    topic_has_lesson_relationship = neo_relationships.TopicIncludesTopicLesson(source=neontology_topic_node, target=neontology_lesson_node)

    # Merge or create relationship in the database
    neon.create_or_merge_neontology_relationship(topic_has_lesson_relationship, database=db_name, operation='merge')
    

In [ ]:
# Create relationships between TopicNode and LearningStatementNode and LessonNode and LearningStatementNode
# Assuming only one node is expected for each query
for learning_statement_node in learning_statement_excel_node_list:
    learning_statement_node_id = learning_statement_node['node'].lesson_learning_statement_id

    learning_statement_nodes = session_tools.find_nodes_by_label_and_properties(driver, 'LearningStatement', {'lesson_learning_statement_id': learning_statement_node_id})
    if not learning_statement_nodes:  # Check if the list is empty
        logging.error(f"No learning statement node found for ID {learning_statement_node_id}")
        continue
    learning_statement_node_properties = dict(learning_statement_nodes[0])
    
    topic_node_id = learning_statement_node['helper']['lesson_learning_statement_topic_id']
    topic_nodes = neon.find_nodes_by_label_and_properties(driver, 'Topic', {'topic_id': topic_node_id})
    if not topic_nodes:  # Check if the list is empty
        logging.error(f"No topic node found for ID {topic_node_id}")
        continue
    topic_node_properties = dict(topic_nodes[0])

    topic_lesson_node_id = learning_statement_node['helper']['lesson_learning_statement_lesson_id']
    topic_lesson_nodes = neon.find_nodes_by_label_and_properties(driver, 'Lesson', {'topic_lesson_id': topic_lesson_node_id})
    if not topic_lesson_nodes:
        logging.error(f"No topic lesson node found for ID {topic_lesson_node_id}")
        continue
    lesson_node_properties = dict(topic_lesson_nodes[0])
    
    # Create instances of TopicNode and TopicLessonNode using the extracted properties
    neontology_lesson_node = neo_curriculum.TopicLessonNode(**lesson_node_properties)
    neontology_topic_node = neo_curriculum.TopicNode(**topic_node_properties)
    neontology_learning_statement_node = neo_curriculum.LearningStatementNode(**learning_statement_node_properties)

    # Create the relationship between learning statement and lesson
    statement_in_lesson_relationship = neo_relationships.LearningStatementPartOfTopicLesson(source=neontology_learning_statement_node, target=neontology_lesson_node)

    # Create the relationship between learning statement and topic
    topic_has_learning_statement_relationship = neo_relationships.TopicIncludesLearningStatement(source=neontology_topic_node, target=neontology_learning_statement_node)

    # Merge or create relationships in the database
    neon.create_or_merge_neontology_relationship(statement_in_lesson_relationship, database=db_name, operation='merge')
    neon.create_or_merge_neontology_relationship(topic_has_learning_statement_relationship, database=db_name, operation='merge')

In [ ]:
# Create relationships between TopicNode and LearningStatementNode and LessonNode and LearningStatementNode
# Assuming only one node is expected for each query
for learning_statement_node in learning_statement_excel_node_list:
    learning_statement_node_id = learning_statement_node['node'].lesson_learning_statement_id

    learning_statement_nodes = session_tools.find_nodes_by_label_and_properties(driver, 'LearningStatement', {'lesson_learning_statement_id': learning_statement_node_id})
    if not learning_statement_nodes:  # Check if the list is empty
        logging.error(f"No learning statement node found for ID {learning_statement_node_id}")
        continue
    learning_statement_node_properties = dict(learning_statement_nodes[0])
    
    topic_node_id = learning_statement_node['helper']['lesson_learning_statement_topic_id']
    topic_nodes = neon.find_nodes_by_label_and_properties(driver, 'Topic', {'topic_id': topic_node_id})
    if not topic_nodes:  # Check if the list is empty
        logging.error(f"No topic node found for ID {topic_node_id}")
        continue
    topic_node_properties = dict(topic_nodes[0])

    topic_lesson_node_id = learning_statement_node['helper']['lesson_learning_statement_lesson_id']
    topic_lesson_nodes = neon.find_nodes_by_label_and_properties(driver, 'Lesson', {'topic_lesson_id': topic_lesson_node_id})
    if not topic_lesson_nodes:
        logging.error(f"No topic lesson node found for ID {topic_lesson_node_id}")
        continue
    lesson_node_properties = dict(topic_lesson_nodes[0])
    
    # Create instances of TopicNode and TopicLessonNode using the extracted properties
    neontology_lesson_node = neo_curriculum.TopicLessonNode(**lesson_node_properties)
    neontology_topic_node = neo_curriculum.TopicNode(**topic_node_properties)
    neontology_learning_statement_node = neo_curriculum.LearningStatementNode(**learning_statement_node_properties)

    # Create the relationship between learning statement and lesson
    statement_in_lesson_relationship = neo_relationships.LearningStatementPartOfTopicLesson(source=neontology_learning_statement_node, target=neontology_lesson_node)

    # Create the relationship between learning statement and topic
    topic_has_learning_statement_relationship = neo_relationships.TopicIncludesLearningStatement(source=neontology_topic_node, target=neontology_learning_statement_node)

    # Merge or create relationships in the database
    neon.create_or_merge_neontology_relationship(statement_in_lesson_relationship, database=db_name, operation='merge')
    neon.create_or_merge_neontology_relationship(topic_has_learning_statement_relationship, database=db_name, operation='merge')

In [ ]:
# Create SchoolNodes from a list
schools = [
    SchoolNode(school_id='FPGS', school_name='Fort Pitt Grammar School', school_org_type='Academy', school_address='Fort Pitt Grammar School, Fort Pitt Hill, Chatham, Kent, ME4 6TJ', school_website='https://www.fortpitt.medway.sch.uk/'),
    SchoolNode(school_id='KCAR', school_name='King Charles School', school_org_type='Academy', school_address='King Charles School, Holly Hall Road, Dudley, West Midlands, DY2 0TZ', school_website='https://www.kingcharlesschool.co.uk/'),
    SchoolNode(school_id='KEV', school_name='Kevlar Academy', school_org_type='Academy', school_address='Kevlar Academy, 123 Fake Street, London, SW1A 1AA', school_website='https://www.kevlaracademy.com/')
]
# Now create a routine to create schools from a list
for school in schools:
    neon.create_or_merge_neontology_node(school, operation='create')

# Create DepartmentNodes from a list
departments = [
    DepartmentNode(department_id='SCI', department_name='Science'),
    DepartmentNode(department_id='MAT', department_name='Mathematics'),
    DepartmentNode(department_id='ENG', department_name='English'),
    DepartmentNode(department_id='HIS', department_name='History')
]
# Now create a routine to create departments from a list
for department in departments:
    neon.create_or_merge_neontology_node(department, operation='create')

# Create KeyStageNodes from a list
key_stages = [
    KeyStageNode(key_stage_level=3, key_stage='KS3'),
    KeyStageNode(key_stage_level=4, key_stage='KS4'),
    KeyStageNode(key_stage_level=5, key_stage='KS5')
]
# Now create a routine to create key stages from a list
for key_stage in key_stages:
    neon.create_or_merge_neontology_node(key_stage, operation='create')
# Create SubjectNodes from a list
subjects = [
    SubjectNode(subject_id='SCI', subject_name='Science'),
    SubjectNode(subject_id='PHY', subject_name='Physics'),
    SubjectNode(subject_id='CHE', subject_name='Chemistry'),
    SubjectNode(subject_id='BIO', subject_name='Biology'),
    SubjectNode(subject_id='MAT', subject_name='Mathematics'),
    SubjectNode(subject_id='ENG', subject_name='English'),
    SubjectNode(subject_id='HIS', subject_name='History')
]
# Now create a routine to create subjects from a list
for subject in subjects:
    neon.create_or_merge_neontology_node(subject, operation='create')


# Create relationships between ALL SchoolNodes and DepartmentNodes
for school_node in schools:
    logging.debug(f"Processing school node: {school_node}")
    school_node_id = school_node.school_id
    logging.pedantic(f"Processing school node ID: {school_node_id}")
    school_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'School', {'school_id': school_node_id})
    logging.pedantic(f"School nodes found: {school_nodes}")
    if not school_nodes:  # Check if the list is empty
        logging.error(f"No school node found for ID {school_node_id}")
        continue
    school_node_properties = dict(school_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"School node properties: {school_node_properties}")

    for department_node in departments:
        logging.debug(f"Processing department node: {department_node}")
        department_node_id = department_node.department_id
        logging.pedantic(f"Processing department node ID: {department_node_id}")
        department_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Department', {'department_id': department_node_id})
        logging.pedantic(f"Department nodes found: {department_nodes}")
        if not department_nodes:  # Check if the list is empty
            logging.error(f"No department node found for ID {department_node_id}")
            continue
        department_node_properties = dict(department_nodes[0])  # Extract properties from the Neo4j Node
        logging.pedantic(f"Department node properties: {department_node_properties}")

        # Create instances of SchoolNode and DepartmentNode using the extracted properties
        logging.pedantic(f"Creating instances of SchoolNode and DepartmentNode")
        neontology_school_node = SchoolNode(**school_node_properties)
        logging.pedantic(f"Neontology school node: {neontology_school_node}")
        neontology_department_node = DepartmentNode(**department_node_properties)
        logging.pedantic(f"Neontology department node: {neontology_department_node}")

        # Create the relationship
        logging.pedantic(f"Creating relationship between {neontology_school_node} and {neontology_department_node}")
        school_has_department_relationship = SchoolHasDepartment(source=neontology_school_node, target=neontology_department_node)
        logging.pedantic(f"Relationship created: {school_has_department_relationship}")

        # Merge or create relationship in the database
        logging.pedantic(f"Merging or creating relationship: {school_has_department_relationship}")
        neon.create_or_merge_neontology_relationship(school_has_department_relationship, operation='merge')
        logging.pedantic(f"Relationship merged or created: {school_has_department_relationship}")

In [ ]:
# Create relationships between DepartmentNodes and SubjectNodes
relationship_list = []
for department_node in departments:
    logging.debug(f"Processing department node: {department_node}")
    department_node_id = department_node.department_id
    logging.pedantic(f"Processing department node ID: {department_node_id}")
    department_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Department', {'department_id': department_node_id})
    logging.pedantic(f"Department nodes found: {department_nodes}")
    if not department_nodes:  # Check if the list is empty
        logging.error(f"No department node found for ID {department_node_id}")
        continue
    department_node_properties = dict(department_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Department node properties: {department_node_properties}")

    for subject_node in subjects:
        logging.debug(f"Processing subject node: {subject_node}")
        subject_node_id = subject_node.subject_id
        logging.pedantic(f"Processing subject node ID: {subject_node_id}")
        subject_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Subject', {'subject_id': subject_node_id})
        logging.pedantic(f"Subject nodes found: {subject_nodes}")
        if not subject_nodes:  # Check if the list is empty
            logging.error(f"No subject node found for ID {subject_node_id}")
            continue
        subject_node_properties = dict(subject_nodes[0])  # Extract properties from the Neo4j Node
        logging.pedantic(f"Subject node properties: {subject_node_properties}")

        # Create instances of DepartmentNode and SubjectNode using the extracted properties
        logging.pedantic(f"Creating instances of DepartmentNode and SubjectNode")
        neontology_department_node = DepartmentNode(**department_node_properties)
        logging.pedantic(f"Neontology department node: {neontology_department_node}")
        neontology_subject_node = SubjectNode(**subject_node_properties)
        logging.pedantic(f"Neontology subject node: {neontology_subject_node}")

        # Create the relationship
        logging.pedantic(f"Creating relationship between {neontology_department_node} and {neontology_subject_node}")
        school_offers_subject_relationship = DepartmentProvidesSubject(source=neontology_department_node, target=neontology_subject_node)
        logging.pedantic(f"Relationship created: {school_offers_subject_relationship}")

        # Merge or create relationship in the database
        logging.pedantic(f"Merging or creating relationship: {school_offers_subject_relationship}")
        neon.create_or_merge_neontology_relationship(school_offers_subject_relationship, operation='merge')
        logging.pedantic(f"Relationship merged or created: {school_offers_subject_relationship}")

In [ ]:
# Create relationships between SubjectNodes, KeyStageNodes and TopicNodes using the excel helper data
relationship_list = []
for topic_node in topic_excel_node_list:
    logging.debug(f"Processing topic node: {topic_node}")
    topic_node_id = topic_node['node'].topic_id
    logging.pedantic(f"Processing topic node ID: {topic_node_id}")
    topic_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Topic', {'topic_id': topic_node_id})
    logging.pedantic(f"Topic nodes found: {topic_nodes}")
    if not topic_nodes:  # Check if the list is empty
        logging.error(f"No topic node found for ID {topic_node_id}")
        continue

    topic_node_properties = dict(topic_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Topic node properties: {topic_node_properties}")
    
    # Get the subject ID and Key Stage from the excel helper data
    subject_node_id = topic_node['helper']['topic_subject']
    key_stage_level = topic_node['helper']['topic_key_stage']

    logging.pedantic(f"Processing subject node ID: {subject_node_id}")
    subject_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Subject', {'subject_id': subject_node_id})
    logging.pedantic(f"Subject nodes found: {subject_nodes}")
    if not subject_nodes:  # Check if the list is empty
        logging.error(f"No subject node found for ID {subject_node_id}")
        continue

    subject_node_properties = dict(subject_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Subject node properties: {subject_node_properties}")

    logging.pedantic(f"Processing key stage level: {key_stage_level}")
    key_stage_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'KeyStage', {'key_stage_level': key_stage_level})
    logging.pedantic(f"Key stage nodes found: {key_stage_nodes}")
    if not key_stage_nodes:  # Check if the list is empty
        logging.error(f"No key stage node found for level {key_stage_level}")
        continue

    key_stage_node_properties = dict(key_stage_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Key stage node properties: {key_stage_node_properties}")

    # Create relationship between the subject and key stage
    logging.pedantic(f"Creating relationship between {subject_node_properties} and {key_stage_node_properties}")
    subject_for_key_stage_relationship = SubjectForKeyStage(source=subject_node_properties, target=key_stage_node_properties)
    logging.pedantic(f"Relationship created: {subject_for_key_stage_relationship}")

    # Create relationship between the topic and subject
    logging.pedantic(f"Creating relationship between {topic_node_properties} and {subject_node_properties}")
    topic_for_subject_relationship = SubjectIncludesTopic(source=subject_node_properties, target=topic_node_properties)
    logging.pedantic(f"Relationship created: {topic_for_subject_relationship}")

    # Create relationship between the topic and key stage
    logging.pedantic(f"Creating relationship between {topic_node_properties} and {key_stage_node_properties}")
    topic_for_key_stage_relationship = KeyStageIncludesTopic(source=key_stage_node_properties, target=topic_node_properties)
    logging.pedantic(f"Relationship created: {topic_for_key_stage_relationship}")

    # Merge or create relationships in the database
    logging.pedantic(f"Merging or creating relationship: {subject_for_key_stage_relationship}")
    neon.create_or_merge_neontology_relationship(subject_for_key_stage_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {subject_for_key_stage_relationship}")

    logging.pedantic(f"Merging or creating relationship: {topic_for_subject_relationship}")
    neon.create_or_merge_neontology_relationship(topic_for_subject_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {topic_for_subject_relationship}")
    
    logging.pedantic(f"Merging or creating relationship: {topic_for_key_stage_relationship}")
    neon.create_or_merge_neontology_relationship(topic_for_key_stage_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {topic_for_key_stage_relationship}")
    
    logging.pedantic(f"Relationships created between {topic_node_properties} and {subject_node_properties} and {key_stage_node_properties}")


In [ ]:
# Create